In [25]:
from dotenv import load_dotenv

load_dotenv()

True

In [26]:
from langchain_teddynote import logging
logging.langsmith("CLASS")

LangSmith 추적을 시작합니다.
[프로젝트명]
CLASS


In [27]:
from langchain_community.document_loaders import PDFPlumberLoader

In [28]:
# pip install pdfplumber

In [ ]:
loader = PDFPlumberLoader('C:\RepoEx\AI_Python_AssistantBot\data\converted_data_test_with_metadata_chap1.pdf')

In [30]:
docs = loader.load()

In [31]:
len(docs)

16

In [32]:
# 문서의 내용 출력
print(docs[10].page_content[:300])

또는 다음과 같이 print 문을 생략하고 변수 이름 a만 입력하여 a의 값을 확인할 수도
있다.
>>> a = "Python"
>>> a
'Python'
조건문 if
다음은 간단한 조건문 if를 사용한 예제이다.
예제는 a가 1보다 크면 "a는 1보다 큽니다."라는 문장을 출력(print)하라는 뜻이다. a는 3
이므로 1보다 크다. 따라서 ‘a는 1보다 큽니다.’라는 문장이 출력된다.
인터프리터 창에서 프롬프트(>>>) 자리에 대신 나타나는 ‘…’은 아직 입력 중인 문장이
끝나지 않았다는 것을 의미한다.
if a > 1: 다음 


In [33]:
docs

[Document(metadata={'source': './data/converted_data_test_with_metadata.pdf', 'file_path': './data/converted_data_test_with_metadata.pdf', 'page': 0, 'total_pages': 16, 'Producer': 'PyPDF2', 'Title': 'Jump to Phython', 'Author': 'Park', 'Subject': 'Python', 'Keywords': 'Python, Book', 'Creator': 'PyPDF2 Library'}, page_content="1장\n01장 파이썬이란 무엇인가?\n필자는 파이썬의 프롬프트(>>>)를 처음 본 순간부터 지금까지 줄곧 파이썬과 함께 지내\n온 듯하다. ‘프로그래밍은 어렵고 지루하다’라는 고정관념을 가지고 있던 필자에게 파이\n썬은 커다란 충격으로 다가왔다. 여러분도 이 책을 통해 파이썬의 매력에 흠뻑 빠져 보\n기를 바란다. 01장에서는 파이썬의 특징과 장단점을 알아보고 파이썬 프로그래밍을 위한\n환경 구축 방법에 대해 배운다. 그리고 간단한 파이썬 프로그램도 작성해 본다.\n01-1 파이썬이란?\n파이썬(Python)은 1990년 암스테르담의 귀도 반 로섬(Guido van rossum)이 개발한 인터\n프리터 언어이다. 귀도는 파이썬이라는 이름을 자신이 좋아하는 코미디 쇼인 ‘몬티 파이\n썬의 날아다니는 서커스(Monty python's flying circus)’에서 따왔다고 한다.\n인터프리터 언어란 소스 코드를 한 줄씩 해석한 후 그때그때 실행해 결과를 바로 확인할\n수 있는 언어를 말한다.\n파이썬의 사전적 의미는 ‘고대 신화에 나오는 파르나소스 산의 동굴에 살던 큰 뱀’을 뜻\n하며, 아폴로 신이 델파이에서 파이썬을 퇴치했다는 이야기가 전해지고 있다. 대부분의\n파이썬 책 표지와 아이콘이 뱀 모양으로 그려져 있는 이유는 바로 이 때문이다.\n파이썬은 컴퓨터 프로그래밍을 교육할 때

In [34]:
for doc in docs:
    doc.metadata['filename'] = doc.metadata['source']

In [35]:
# !pip install -q ragas==0.1.19

In [36]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.extractor import KeyphraseExtractor
from ragas.testset.docstore import InMemoryDocumentStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [37]:
# 생성기
generator_llm = ChatOpenAI(model = 'gpt-4o-mini')

In [38]:
# 평가를 할 수 있는 비평기
critic_llm = ChatOpenAI(model = 'gpt-4o-mini')

In [39]:
# 임베딩 모델
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')

In [40]:
# 텍스트 분할기
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)

In [41]:
langchain_llm = LangchainLLMWrapper(ChatOpenAI(model = 'gpt-4o-mini'))
# 구문추출기 생성을 위한 모델 호출(RAGAS와의 호환을 위한 Wrapper)

In [42]:
# 구문추출기 : 문서 핵심 정보 식별 및 추출 역할
Keyphrase_extractor = KeyphraseExtractor(llm = langchain_llm)

In [43]:
# 임베딩 모델도 RAGAS와 호환을 위해 Wrapper 적용
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

In [44]:
# 저장소, docstore 만들기
docstore = InMemoryDocumentStore(
    splitter = splitter,
    embeddings = ragas_embeddings,
    extractor = Keyphrase_extractor
)

In [45]:
# 생성기 만들기
generator = TestsetGenerator.from_langchain(
    generator_llm, # 생성기
    critic_llm,  # 판별기
    ragas_embeddings, # 임베딩모델
    docstore = docstore # 문서저장소
)

In [46]:
# # 질문 유형 분포를 설정하는 딕셔너리입니다.
# # 각 질문 유형과 그에 해당하는 비율(0~1)을 정의합니다.
# distributions = {
#     "simple": 0.4,          # 단순한 질문 유형의 비율 (전체의 40%)
#     "reasoning": 0.2,       # 논리적 사고가 필요한 질문 유형의 비율 (전체의 20%)
#     "multi_context": 0.2,   # 여러 문맥을 고려해야 하는 질문 유형의 비율 (전체의 20%)
#     "conditional": 0.2      # 조건에 따라 답변이 달라지는 질문 유형의 비율 (전체의 20%)
# }

In [198]:
# 질문 유형 분포 결정
distributions = {simple: 0.6, reasoning: 0.1, multi_context: 0.1, conditional: 0.2}
# distributions = {simple : 0.4, reasoning : 0.2, multi_context : 0.2, conditional : 0.2}

In [ ]:
# def batch_documents(documents, batch_size):
#     """
#     문서를 일정 크기의 배치로 나눕니다.
    
#     Args:
#         documents (list): 문서 목록.
#         batch_size (int): 각 배치의 크기.
    
#     Returns:
#         list: 배치로 나뉜 문서 그룹.
#     """
#     for i in range(0, len(documents), batch_size):
#         yield documents[i:i + batch_size]

# def generate_testset_in_batches(generator, documents, test_size, distributions, batch_size=10):
#     """
#     문서를 배치 단위로 나누어 처리하여 테스트 세트를 생성합니다.
    
#     Args:
#         generator: 테스트 세트 생성기 객체.
#         documents (list): 전체 문서 목록.
#         test_size (int): 각 배치에서 생성할 질문 세트 크기.
#         distributions (dict): 질문 유형 분포.
#         batch_size (int): 배치 크기 (기본값: 10).
    
#     Returns:
#         list: 각 배치의 결과를 결합한 전체 테스트 세트.
#     """
#     all_testsets = []
#     for batch_index, document_batch in enumerate(batch_documents(documents, batch_size)):
#         print(f"Processing batch {batch_index + 1}/{len(documents) // batch_size + 1}...")
#         try:
#             batch_testset = generator.generate_with_langchain_docs(
#                 documents=document_batch,
#                 test_size=test_size,
#                 distributions=distributions,
#                 with_debugging_logs=True,
#                 raise_exceptions=False
#             )
#             all_testsets.extend(batch_testset.to_dict())  # 배치 결과를 결합
#         except Exception as e:
#             print(f"Error processing batch {batch_index + 1}: {e}")
    
#     return all_testsets

# # 예제 사용
# testset = generate_testset_in_batches(
#     generator=generator,
#     documents=docs,  # 전체 문서
#     test_size=10,    # 각 배치에서 생성할 테스트 세트 크기
#     distributions=distributions,
#     batch_size=50    # 배치 크기
# )

# # 결과를 pandas DataFrame으로 변환
# import pandas as pd
# test_df = pd.DataFrame(testset)

# # DataFrame 확인
# print(test_df.head())


Processing batch 1/1...


embedding nodes:  14%|█▎        | 6/44 [00:01<00:04,  7.77it/s][ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Extension', 'Plugin', 'Browser', 'Usage', 'Functionality']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Extension', 'Plugin', 'Browser', 'Usage', 'Functionality']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Extension', 'Plugin', 'Browser', 'Usage', 'Functionality']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Extension', 'Plugin', 'Browser', 'Usage', 'Functionality']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Extension', 'Plugin', 'Browser', 'Usage', 'Functionality']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Extension', 'Plugin', 'Browser', 'Usage', 'Functionality']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Extension', 'Plugin', 'Browser', 'Usage', 'Functionality']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Extension', 'Plugin', 'Browser', 'Usage', 'Functionality']}
[

Error processing batch 1: 'TestDataset' object has no attribute 'to_dict'
Empty DataFrame
Columns: []
Index: []


In [56]:
testset = generator.generate_with_langchain_docs(
    documents = docs,
    test_size = 10,
    distributions = distributions,
    with_debugging_logs = True,
    # 로그 활성화
    raise_exceptions= False
    # 예외가 발생해도 계속 진행, 예외 발생 시 문서가 처리 되지 않거나 문서에 기록됨
)

embedding nodes:  23%|██▎       | 10/44 [00:00<00:01, 19.18it/s][ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['프로그램', '플랫폼', '사용자 경험', '편리한 접근', '기능']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['프로그램', '플랫폼', '사용자 경험', '편리한 접근', '기능']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['프로그램', '플랫폼', '사용자 경험', '편리한 접근', '기능']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['프로그램', '플랫폼', '사용자 경험', '편리한 접근', '기능']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['프로그램', '플랫폼', '사용자 경험', '편리한 접근', '기능']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['프로그램', '플랫폼', '사용자 경험', '편리한 접근', '기능']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['프로그램', '플랫폼', '사용자 경험', '편리한 접근', '기능']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['프로그램', '플랫폼', '사용자 경험', '편리한 접근', '기능']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['프로그램', '플랫폼', '사용자 경험', '편리한 접근', '기능']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases':

In [57]:
test_df = testset.to_pandas()
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the purpose of using 'import sys' in P...,[위와 같은 창을 ‘파이썬 대화형 인터프리터’라고 부른다. 여기서 인터프리터란 사용...,The answer to given question is not present in...,simple,[{'source': './data/converted_data_test_with_m...,True
1,What is the significance of Visual Studio Code...,[명령 프롬프트 창이 열린다.\n첫 번째 줄에는 다음과 같은 프롬프트가 나타나 있을...,The answer to given question is not present in...,simple,[{'source': './data/converted_data_test_with_m...,True
2,How does prompt input affect command execution...,[위와 같은 창을 ‘파이썬 대화형 인터프리터’라고 부른다. 여기서 인터프리터란 사용...,The context does not provide information on ho...,reasoning,[{'source': './data/converted_data_test_with_m...,True
3,How do IDLE's features relate to Python progra...,[명령 프롬프트 창이 열린다.\n첫 번째 줄에는 다음과 같은 프롬프트가 나타나 있을...,The context does not provide specific informat...,reasoning,[{'source': './data/converted_data_test_with_m...,True
4,How does ML boost AI capabilities?,[o 시스템과 밀접한 프로그래밍 영역\no 모바일 프로그래밍\n파이썬으로 할 수 있...,The context does not provide specific informat...,reasoning,[{'source': './data/converted_data_test_with_m...,True
5,What can you run in Python's interactive shell?,[위와 같은 창을 ‘파이썬 대화형 인터프리터’라고 부른다. 여기서 인터프리터란 사용...,"In Python's interactive shell, you can run Pyt...",multi_context,[{'source': './data/converted_data_test_with_m...,True
6,What can IDLE do for defining and running func...,[함수\ndef는 파이썬에서 함수를 정의할 때 사용하는 예약어이다. 위 예제는 ad...,IDLE can be used to define functions by writin...,multi_context,[{'source': './data/converted_data_test_with_m...,True
7,"Which command navigates directories in CMD, an...",[상 .py로 해야 한다. .py는 파이썬 파일이라는 것을 알려 주는 확장자이다.\...,The command that navigates directories in CMD ...,multi_context,[{'source': './data/converted_data_test_with_m...,True
8,What basic Python cmds for simple data manipul...,[위와 같은 창을 ‘파이썬 대화형 인터프리터’라고 부른다. 여기서 인터프리터란 사용...,The context does not provide specific basic Py...,conditional,[{'source': './data/converted_data_test_with_m...,True
9,How to run a Python script from another direct...,[명령 프롬프트 창이 열린다.\n첫 번째 줄에는 다음과 같은 프롬프트가 나타나 있을...,"To run a Python script from another directory,...",conditional,[{'source': './data/converted_data_test_with_m...,True


In [58]:
df = test_df

In [59]:
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the purpose of using 'import sys' in P...,[위와 같은 창을 ‘파이썬 대화형 인터프리터’라고 부른다. 여기서 인터프리터란 사용...,The answer to given question is not present in...,simple,[{'source': './data/converted_data_test_with_m...,True
1,What is the significance of Visual Studio Code...,[명령 프롬프트 창이 열린다.\n첫 번째 줄에는 다음과 같은 프롬프트가 나타나 있을...,The answer to given question is not present in...,simple,[{'source': './data/converted_data_test_with_m...,True
2,How does prompt input affect command execution...,[위와 같은 창을 ‘파이썬 대화형 인터프리터’라고 부른다. 여기서 인터프리터란 사용...,The context does not provide information on ho...,reasoning,[{'source': './data/converted_data_test_with_m...,True
3,How do IDLE's features relate to Python progra...,[명령 프롬프트 창이 열린다.\n첫 번째 줄에는 다음과 같은 프롬프트가 나타나 있을...,The context does not provide specific informat...,reasoning,[{'source': './data/converted_data_test_with_m...,True
4,How does ML boost AI capabilities?,[o 시스템과 밀접한 프로그래밍 영역\no 모바일 프로그래밍\n파이썬으로 할 수 있...,The context does not provide specific informat...,reasoning,[{'source': './data/converted_data_test_with_m...,True


In [60]:
from datasets import Dataset

test_dataset = Dataset.from_pandas(df)
test_dataset

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done'],
    num_rows: 10
})

In [61]:
df['contexts'][0]

['위와 같은 창을 ‘파이썬 대화형 인터프리터’라고 부른다. 여기서 인터프리터란 사용자가\n입력한 소스 코드를 실행하는 환경을 뜻한다.\n파이썬 대화형 인터프리터를 간단히 ‘파이썬 셸(python shell)’이라고도 부른다. 파이썬 셸\n창에서 3개의 꺾은 괄호(>>>)를 프롬프트(prompt)라고 부른다. 프롬프트(>>>)가 있는\n곳이 입력하는 부분이다.\n그렇다면 ‘대화형’은 무슨 의미일까?\n입력에 따른 결괏값이 바로 출력된다. 이렇게 입출력이 번갈아 이어지는 것이 마치 대화\n하는 것처럼 느껴지기 때문에 파이썬 대화형 인터프리터라고 부른다. 대화형 인터프리터\n를 사용하면 간단한 예제를 풀 때 결과를 바로 확인할 수 있어 학습을 하는 데 효과적이\n다. 앞으로 파이썬의 기본적인 내용은 이 창을 사용해 실습한다.\n대화형 인터프리터를 종료할 때는 [Ctrl+Z]를 누른 후 [Enter] 를 누른다(유닉스 계열에서\n는 [Ctrl+D]). 또는 다음과 같이 내장 함수인 quit()이나 exit()를 사용하여 종료한다.\n다음 예와 같이 sys 모듈을 사용하여 종료할 수도 있다.\nimport sys\nsys.exit()\n모듈을 사용하는 방법은 05장을 참고하기 바란다\n파이썬 기초 문법 따라 해 보기\n이제 파이썬 대화형 인터프리터(파이썬 셸)를 실행하여 파이썬 기초 문법을 따라 해 보\n자. 여기에서 소개하는 내용은 뒤에서 자세하게 다룰 것이므로 지금 당장 이해되지 않는\n다고 해서 절망하거나 고심하지는 말자.\n사칙 연산\n1 더하기(+) 2는 3이라는 값을 출력해 보자. 보통 계산기를 사용하듯이 +만 넣어 주면\n된다.',
 '위와 같은 창을 ‘파이썬 대화형 인터프리터’라고 부른다. 여기서 인터프리터란 사용자가\n입력한 소스 코드를 실행하는 환경을 뜻한다.\n파이썬 대화형 인터프리터를 간단히 ‘파이썬 셸(python shell)’이라고도 부른다. 파이썬 셸\n창에서 3개의 꺾은 괄호(>>>)를 프롬프트(prompt)라고 부른다. 프롬프트(>>>)

In [62]:
import ast
def convert_to_list(example):
    contexts = ast.literal_eval(example['contexts'])
    return {'contexts':contexts}

In [155]:
# test_dataset = test_dataset.map(convert_to_list)

In [156]:
test_dataset[0]['contexts']

['위와 같은 창을 ‘파이썬 대화형 인터프리터’라고 부른다. 여기서 인터프리터란 사용자가\n입력한 소스 코드를 실행하는 환경을 뜻한다.\n파이썬 대화형 인터프리터를 간단히 ‘파이썬 셸(python shell)’이라고도 부른다. 파이썬 셸\n창에서 3개의 꺾은 괄호(>>>)를 프롬프트(prompt)라고 부른다. 프롬프트(>>>)가 있는\n곳이 입력하는 부분이다.\n그렇다면 ‘대화형’은 무슨 의미일까?\n입력에 따른 결괏값이 바로 출력된다. 이렇게 입출력이 번갈아 이어지는 것이 마치 대화\n하는 것처럼 느껴지기 때문에 파이썬 대화형 인터프리터라고 부른다. 대화형 인터프리터\n를 사용하면 간단한 예제를 풀 때 결과를 바로 확인할 수 있어 학습을 하는 데 효과적이\n다. 앞으로 파이썬의 기본적인 내용은 이 창을 사용해 실습한다.\n대화형 인터프리터를 종료할 때는 [Ctrl+Z]를 누른 후 [Enter] 를 누른다(유닉스 계열에서\n는 [Ctrl+D]). 또는 다음과 같이 내장 함수인 quit()이나 exit()를 사용하여 종료한다.\n다음 예와 같이 sys 모듈을 사용하여 종료할 수도 있다.\nimport sys\nsys.exit()\n모듈을 사용하는 방법은 05장을 참고하기 바란다\n파이썬 기초 문법 따라 해 보기\n이제 파이썬 대화형 인터프리터(파이썬 셸)를 실행하여 파이썬 기초 문법을 따라 해 보\n자. 여기에서 소개하는 내용은 뒤에서 자세하게 다룰 것이므로 지금 당장 이해되지 않는\n다고 해서 절망하거나 고심하지는 말자.\n사칙 연산\n1 더하기(+) 2는 3이라는 값을 출력해 보자. 보통 계산기를 사용하듯이 +만 넣어 주면\n된다.',
 '위와 같은 창을 ‘파이썬 대화형 인터프리터’라고 부른다. 여기서 인터프리터란 사용자가\n입력한 소스 코드를 실행하는 환경을 뜻한다.\n파이썬 대화형 인터프리터를 간단히 ‘파이썬 셸(python shell)’이라고도 부른다. 파이썬 셸\n창에서 3개의 꺾은 괄호(>>>)를 프롬프트(prompt)라고 부른다. 프롬프트(>>>)

In [157]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [158]:
loader = PyMuPDFLoader('data/converted_data_with_metadata.pdf')
docs = loader.load()

In [144]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 50)
split_documents = text_splitter.split_documents(docs)

In [159]:
embeddings = OpenAIEmbeddings()

In [160]:
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [161]:
retriever = vectorstore.as_retriever()

In [ ]:
prompt = PromptTemplate.from_template(
    """당신은 질문에 답변을 수행하는 파이썬 관련 용어 사전을 제공하는 학습 비서입니다. 
    당신은 주어진 context에서 주어진 question에 답하는 것을 수행합니다. 검색된 결과인 다음 context를 사용하여 질문인 question에 답하세요.
    만약, context에서 답을 찾을 수 없거나, 답을 모른다면 '주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.'라고 한글로 질문하고 답변해 주세요.
    이름이나 기술적인 용어는 번역하지 않고 그대로 출력해주세요.
    
    
    # Context : {context}
    # Question : {question}
    # Answer :
    """
)

In [195]:
llm = ChatOpenAI(model = 'gpt-4o-mini', temperature = 0)

In [ ]:
chain = (
    {'context' : retriever, 'question' : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
batch_dataset = []
for question in test_dataset['question']:
    batch_dataset.append(question)

KeyError: "Column 질문 not in the dataset. Current columns in the dataset: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done', 'answer']"

In [192]:
batch_dataset[:3]

["What is the purpose of using 'import sys' in Python programming?",
 'What is the significance of Visual Studio Code in programming environments?',
 'How does prompt input affect command execution and program flow?']

In [193]:
answer = chain.batch(batch_dataset)
answer[:3]

["'주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.'",
 '주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.',
 '주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.']

In [123]:
if 'answer' in test_dataset.column_names:
    test_dataset = test_dataset.remove_columns(['answer']).add_column('answer', answer)
else:
    test_dataset = test_dataset.add_column('answer', answer)

In [124]:
test_dataset

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done', 'answer'],
    num_rows: 10
})

In [125]:
from ragas import evaluate
from ragas.metrics import answer_relevancy,faithfulness,context_recall,context_precision

In [126]:
result = evaluate(
    dataset = test_dataset,
    metrics = [
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision
    ]
)


# answer_relevancy : 모델이 문맥에서 중요한 정보를 잘 회상, 재현했는가
# faithfulness : 모델의 답변이 문맥에 기반해서 사실적인가(factual)
# context_recall : 전체 질문에 대해 모델의 답변이 질문과 얼마나 관련성이 있는가
# context_precision : 모델이 문맥에서 필요한 정보를 정확히 활용했는가

Evaluating: 100%|██████████| 40/40 [00:12<00:00,  3.16it/s]


In [127]:
result

{'answer_relevancy': 0.0863, 'faithfulness': 0.0889, 'context_recall': 0.8667, 'context_precision': 0.4000}

In [128]:
result_df = result.to_pandas()
result_df

,question,contexts,answer,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,What is the purpose of using 'import sys' in P...,[위와 같은 창을 ‘파이썬 대화형 인터프리터’라고 부른다. 여기서 인터프리터란 사용...,주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.,The answer to given question is not present in...,0.000000,0.000000,1.000000,0.0
1,What is the significance of Visual Studio Code...,[명령 프롬프트 창이 열린다.\n첫 번째 줄에는 다음과 같은 프롬프트가 나타나 있을...,비주얼 스튜디오 코드(Visual Studio Code)는 파이참(Pycharm)과...,The answer to given question is not present in...,0.863301,0.888889,1.000000,0.0
2,How does prompt input affect command execution...,[위와 같은 창을 ‘파이썬 대화형 인터프리터’라고 부른다. 여기서 인터프리터란 사용...,주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.,The context does not provide information on ho...,0.000000,0.000000,1.000000,0.0
3,How do IDLE's features relate to Python progra...,[명령 프롬프트 창이 열린다.\n첫 번째 줄에는 다음과 같은 프롬프트가 나타나 있을...,주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.,The context does not provide specific informat...,0.000000,0.000000,1.000000,0.0
4,How does ML boost AI capabilities?,[o 시스템과 밀접한 프로그래밍 영역\no 모바일 프로그래밍\n파이썬으로 할 수 있...,주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.,The context does not provide specific informat...,0.000000,0.000000,1.000000,0.0
5,What can you run in Python's interactive shell?,[위와 같은 창을 ‘파이썬 대화형 인터프리터’라고 부른다. 여기서 인터프리터란 사용...,주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.,"In Python's interactive shell, you can run Pyt...",0.000000,0.000000,0.000000,1.0
6,What can IDLE do for defining and running func...,[함수\ndef는 파이썬에서 함수를 정의할 때 사용하는 예약어이다. 위 예제는 ad...,주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.,IDLE can be used to define functions by writin...,0.000000,0.000000,1.000000,1.0
7,"Which command navigates directories in CMD, an...",[상 .py로 해야 한다. .py는 파이썬 파일이라는 것을 알려 주는 확장자이다.\...,주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.,The command that navigates directories in CMD ...,0.000000,0.000000,0.666667,1.0
8,What basic Python cmds for simple data manipul...,[위와 같은 창을 ‘파이썬 대화형 인터프리터’라고 부른다. 여기서 인터프리터란 사용...,주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.,The context does not provide specific basic Py...,0.000000,0.000000,1.000000,0.0
9,How to run a Python script from another direct...,[명령 프롬프트 창이 열린다.\n첫 번째 줄에는 다음과 같은 프롬프트가 나타나 있을...,주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.,"To run a Python script from another directory,...",0.000000,0.000000,1.000000,1.0


In [129]:
result_df.loc[:, 'context_recall':'context_precision']

,context_recall,context_precision
0,1.000000,0.0
1,1.000000,0.0
2,1.000000,0.0
3,1.000000,0.0
4,1.000000,0.0
5,0.000000,1.0
6,1.000000,1.0
7,0.666667,1.0
8,1.000000,0.0
9,1.000000,1.0


|context_precision (문맥 정밀도)|faithfulness (신뢰성)|answer_relevancy (답변 적합성)|context_recall (문맥 재현율)|
|---|---|---|---|
|답변에 사용된 문맥이 정확했는지|답변이 문맥에 기반해 사실적(factual)인지|답변이 질문과 관련성이 있는지|답변이 문맥에서 얼마나 많은 정보를 활용했는지|
|1.0: 답변에 사용된 정보가 문맥에서 매우 정확하게 활용됨|1.0: 답변이 문맥과 매우 정확하게 일치|1.0: 답변이 질문과 매우 높은 관련성을 가짐|1.0: 답변이 문맥의 모든 중요한 정보를 잘 회상|
|0.0: 답변이 문맥과 무관하거나, 부정확한 정보를 활용|0.0: 답변이 문맥에 기반하지 않고, 사실과 다를 가능성이 큼|0.0: 답변이 질문과 관련이 없거나, 부정확|0.0: 답변이 문맥을 활용하지 못함|